# Semantic Search

In [4]:
import re

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F

from razdel import sentenize

from transformers import AutoTokenizer, AutoModel

from tqdm.notebook import tqdm

### Data Preprocessing

В качестве исходного текста будем использовать роман-антиутопию Джорджа Оруэлла "1984": 

In [2]:
with open('1984.txt', 'r', encoding='utf-8') as f:
    src_text = f.read()

src_text[:100]

'Был холодный ясный апрельский день, и часы пробили тринадцать. Уткнув подбородок в грудь, чтобы спас'

Разобьем текст на сегменты:

In [9]:
SEGMENT_LENGTH = 5 # sentences in 1 segment
HYPERLINK_PATTERN = re.compile(r'\([0-9]+\)')

In [10]:
def preprocess_sentence(sentence):
    """Strips the sentence and removes (i)-type links from it."""
    sentence = sentence.text.strip()
    
    start = 0
    clean_sentence = []
    for match in HYPERLINK_PATTERN.finditer(sentence):
        end = match.start()
    
        clean_sentence.append(sentence[start:end])
    
        start = match.end()
    
    clean_sentence.append(sentence[start:])
    return ''.join(clean_sentence)

In [11]:
sentences = sentenize(src_text)

segments = []
segment = []
for sentence in sentences:
    # clean each sentence
    clean_sentence = preprocess_sentence(sentence)

    # add sentence to the segment
    segment.append(clean_sentence)
    if len(segment) == SEGMENT_LENGTH:
        segments.append(' '.join(segment))
        segment = []

len(segments)

1316

In [12]:
segments[0], segments[-1]

('Был холодный ясный апрельский день, и часы пробили тринадцать. Уткнув подбородок в грудь, чтобы спастись от злого ветра, Уинстон Смит торопливо шмыгнул за стеклянную дверь жилого дома «Победа», но все-таки впустил за собой вихрь зернистой пыли. В вестибюле пахло вареной капустой и старыми половиками. Против входа на стене висел цветной плакат, слишком большой для помещения. На плакате было изображено громадное, больше метра в ширину, лицо, — лицо человека лет сорока пяти, с густыми черными усами, грубое, но по-мужски привлекательное.',
 'Полным переводом мог стать бы только идеологический перевод, в котором слова Джефферсона превратились бы в панегирик абсолютной власти. Именно таким образом и переделывалась, кстати, значительная часть литературы прошлого. Из престижных соображений было желательно сохранить память о некоторых исторических лицах, в то же время приведя их труды в согласие с учением ангсоца. Уже шла работа над переводом таких писателей, как Шекспир, Мильтон, Свифт, Байр

Добавим к каждому сегменту префикс:

In [13]:
prefixes = ('query: ', 'passage: ')
prefixed_segments = []

for segment in segments:
    prefixed_segments.append(prefixes[1] + segment)

prefixed_segments[0], prefixed_segments[-1]

('passage: Был холодный ясный апрельский день, и часы пробили тринадцать. Уткнув подбородок в грудь, чтобы спастись от злого ветра, Уинстон Смит торопливо шмыгнул за стеклянную дверь жилого дома «Победа», но все-таки впустил за собой вихрь зернистой пыли. В вестибюле пахло вареной капустой и старыми половиками. Против входа на стене висел цветной плакат, слишком большой для помещения. На плакате было изображено громадное, больше метра в ширину, лицо, — лицо человека лет сорока пяти, с густыми черными усами, грубое, но по-мужски привлекательное.',
 'passage: Полным переводом мог стать бы только идеологический перевод, в котором слова Джефферсона превратились бы в панегирик абсолютной власти. Именно таким образом и переделывалась, кстати, значительная часть литературы прошлого. Из престижных соображений было желательно сохранить память о некоторых исторических лицах, в то же время приведя их труды в согласие с учением ангсоца. Уже шла работа над переводом таких писателей, как Шекспир, Ми

### Segment Vectorization

Векторизуем каждый сегмент с помощью модели E5:

In [14]:
checkpoint = 'intfloat/multilingual-e5-large'

In [15]:
def average_pool(last_hidden_states, attention_mask):
    """Averages hidden states from the last layer."""
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [16]:
def batch_sample(texts, batch_size=16):
    """Splits the texts into batches."""
    batches = []

    for i in range(0, len(texts), batch_size):
        batches.append(texts[i: i + batch_size])

    return batches

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [18]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [19]:
batches = batch_sample(prefixed_segments, batch_size=4)
passage_embeddings = []

for batch in tqdm(batches, desc='Vectorizing batches'):
    # Tokenize the input batch
    batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)

    # calculate embeddings
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    del outputs
    
    # normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1).detach().to('cpu').numpy()
    passage_embeddings.append(embeddings)

Vectorizing batches:   0%|          | 0/329 [00:00<?, ?it/s]

Сохраним по отдельности отрывки и их эмбеддинги:

In [22]:
passage_embeddings = np.vstack(passage_embeddings)

In [23]:
passages_df = pd.DataFrame(zip(segments, passage_embeddings), columns=['Отрывок', 'Эмбеддинг'])
passages_df.to_pickle('passages_db.pkl')

### Search Through The Database

Реализуем поиск по собранной БД.

In [24]:
passages_df = pd.read_pickle('passages_db.pkl')

passages_df

,Отрывок,Эмбеддинг
0,"Был холодный ясный апрельский день, и часы про...","[0.02479827217757702, -0.015230799093842506, -..."
1,Уинстон направился к лестнице. К лифту не стои...,"[0.039334312081336975, -0.02482466958463192, -..."
2,На каждой площадке со стены глядело все то же ...,"[0.03751745447516441, -0.015397601760923862, -..."
3,"Уинстон повернул ручку, голос ослаб, но речь п...","[0.040765102952718735, -0.011668235994875431, ..."
4,Ветер закручивал спиралями пыль и обрывки бума...,"[0.018811898306012154, -0.006494827102869749, ..."
...,...,...
1311,"В сущности, использовать новояз для неортодокс...","[0.032024458050727844, 0.012103094719350338, -..."
1312,На практике любому воспитанному в двоемыслии и...,"[0.020894888788461685, -0.007327871397137642, ..."
1313,"История уже была переписана, но фрагменты стар...","[0.018215926364064217, -0.02274288795888424, 0..."
1314,"Возьмем, например, хорошо известный отрывок из...","[0.015750784426927567, -0.007353644352406263, ..."


In [35]:
idx_to_embs = {k: v for k, v in enumerate(passages_df['Эмбеддинг'])}
idx_to_passages = {k: v for k, v in enumerate(passages_df['Отрывок'])}

Приведем несколько примеров запросов:

In [28]:
queries = [
    'Что такое ангсоц?',
    'Кто является главным злодеем произведения?',
    'Перечисли основные министерства Англии.',
    'Расскажи о Джулии.'
]

Будем оценивать косинусную близость запросов с сегментами:

In [32]:
def cosine_sim(v1, v2):
    """Calculates cosine similarity between two 1-d vectors."""
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [41]:
def select_best_segments(query, top_k=5):
    """Selects top-k segments based on cosine similarity with the query."""
    # prefix the query and calculate its embedding
    prefixed_query = prefixes[0] + query
    query_dict = tokenizer(
        prefixed_query, 
        max_length=512, 
        padding=True, 
        truncation=True, 
        return_tensors='pt'
    ).to(device)
    query_embedding = F.normalize(
        average_pool(
            model(**query_dict).last_hidden_state,
            query_dict['attention_mask'],
        ),
        p=2,
        dim=1,
    ).detach().to('cpu').numpy().squeeze(0)
    
    # rank candidate segments
    candidates = [(idx, emb) for idx, emb in idx_to_embs.items()]
    top_candidates = sorted(
        candidates, reverse=True, key=lambda x: cosine_sim(x[1], query_embedding)
    )[:top_k]
    
    # form the ranked output with statistics
    output = []
    for i, elem in enumerate(top_candidates):
        output.append((cosine_sim(elem[1], query_embedding), idx_to_passages[elem[0]]))
    
    return output

In [42]:
for query in queries:
    print(f'Query: {query}')
    print('=' * 90)
    print(f'Best candidates:')
    
    best_segments = select_best_segments(query)
    
    for i, elem in enumerate(best_segments):
        print(f'{i+1}. {elem[0]} -> {elem[1]}')
        
    print()

Query: Что такое ангсоц?
Best candidates:
1. 0.818052377531268 -> Где-то вдалеке с глухим раскатистым грохотом разорвалась ракета. Теперь их падало на Лондон по двадцать-тридцать штук в неделю. Внизу на улице ветер трепал рваный плакат, на нем мелькало слово АНГСОЦ. Ангсоц. Священные устои ангсоца.
2. 0.8175249410601317 -> Возьмем, например, типичное предложение из передовой статьи в «Таймс»: «Старомыслы не нутрят ангсоц». Кратчайшим образом на староязе это можно изложить так: «Те, чьи идеи сложились до Революции, не воспринимают всей душой принципов английского социализма». Но это неадекватный перевод. Во-первых, чтобы как следует понять смысл приведенной фразы, надо иметь четкое представление о том, что означает слово «ангсоц». Кроме того, лишь человек, воспитанный в ангсоце, почувствует всю силу слова «нутрить», подразумевающего слепое восторженное приятие, которое в наши дни трудно вообразить, или слова «старомысл», неразрывно связанного с понятиями порока и вырождения.
3. 0.817317

### Conclusion

Система поиска на основе E5, в целом, работает неплохо. Например, top-1 ответ в вопросе про министерства выдает "официальную" информацию о них с точки зрения обыкновенного работника Англии, а top-4 показывает, что названия министерств противоположны их реальной деятельности.

Однако, далеко не всегда удается уловить такую "игру смыслов". В вопросе про главного злодея модель только в одной выжимке (top-2) выдает достаточно близкую к истине информацию (Большой Брат и текущий строй в целом), в остальных же злодеем выступает Голдстейн (что на деле не так), и ни разу не упоминается скрытый до последнего антагонист - О'Брайен.

Более того, в коротких вопросах модель сильно зацикливается на отдельных словах. Так, в запросе про Джулию система не выдала желаемую инфоормацию, а просто привела диалоги, где имя Джулии встречалось чаще всего.

Если информации по вопросу в целом мало (ангсоц), то и выдача модели будет малоинформативной.

В итоге, с точки зрения чистого информационного поиска система +- справляется со своей задачей, но выделение истинно близких по семантике предложений хромает.